In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
pd.__version__ # get the version of pandas

In [ ]:
df_raw=pd.read_csv('/Users/francois-guillaume.rideau/Documents/Kaggle/Kaggle-SF/train.csv')
df_test_raw=pd.read_csv('/Users/francois-guillaume.rideau/Documents/Kaggle/Kaggle-SF/test.csv')

In [ ]:
print(df_raw.shape)
print(df_test_raw.shape)
Nobs_train_raw = df_raw.shape[0]
Nobs_train_raw
df_raw['Id']=range(Nobs_train_raw)

In [ ]:
minidf=df_raw.loc[0:5,:]
minidf
minidf_test=df_test_raw.loc[0:5,:]
minidf_test

In [ ]:
df_raw.describe() # shows one value for Y which is totally stupid

In [ ]:
df_raw[df_raw['Y']>38] # note that X is -120.5 for all those obs. We still have PdDistrict and Address...67 such OBS

In [ ]:
df_raw.isnull().values.any() # no missing values

In [ ]:
df_test_raw.describe() # shows one value for Y which is totally stupid

In [ ]:
df_test_raw[df_test_raw['Y']>38] # note that X is -120.5 for all those obs. We still have PdDistrict and Address...76 such OBS

In [ ]:
df_test_raw.isnull().values.any() # no missing values

In [ ]:
# converting columns to the proper format (training set)

df_raw['Dates']=pd.to_datetime(df_raw['Dates'])

def is_categorical_a(array_like):
    return array_like.dtype.name == 'category'

for col in ['Category', 'DayOfWeek', 'PdDistrict', 'Resolution']:
    df_raw[col] = df_raw[col].astype('category')
    
df_raw.dtypes

# converting columns to the proper format (test set)

df_test_raw['Dates']=pd.to_datetime(df_test_raw['Dates'])

for col in ['DayOfWeek', 'PdDistrict']:
    df_test_raw[col] = df_test_raw[col].astype('category')
    
df_test_raw.dtypes
df_t = df_test_raw

In [ ]:
# create a clean dataframe for the training test

df=df_raw[df_raw['Y']<=38]

# explore the dataframe
df_raw['PdDistrict'].describe()
df_raw.dtypes

In [ ]:
df.loc[:,'Hour']=df['Dates'].dt.hour
df.loc[:,'Month']=df['Dates'].dt.month
type(df.loc)
# df['Hour']

df_t.loc[:,'Hour']=df_t['Dates'].dt.hour
df_t.loc[:,'Month']=df_t['Dates'].dt.month
type(df_t.loc)
df

In [ ]:
# example how to make dummy variables
s= pd.Series(["Wednesday","Tuesday","Friday","Thursday","Saturday","Sunday","Monday"], dtype="category")
s = s.cat.reorder_categories(["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"], ordered=True)
pd.get_dummies(s, drop_first=True)

df.loc[:,'DayOfWeek'] = df['DayOfWeek'].cat.reorder_categories(["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"], ordered=True)
x=pd.get_dummies(df['DayOfWeek'], drop_first=True)
y=pd.get_dummies(df['PdDistrict'], drop_first=True)

In [ ]:
# df['Hour'].describe() # 0 to 23

def hourslice2 (hour):
    if   (hour==0)  or (hour==23) : return ("23H - 0H")
    elif (hour==1)  or (hour==2)  : return (" 1H - 2H")
    elif (hour==3)  or (hour==4)  : return (" 3H - 4H")
    elif (hour==5)  or (hour==6)  : return (" 5H - 6H")
    elif (hour==7)  or (hour==8)  : return (" 7H - 8H")
    elif (hour==9)  or (hour==10) : return (" 9H - 10H")
    elif (hour==11) or (hour==12) : return ("11H - 12H")
    elif (hour==13) or (hour==14) : return ("13H - 14H")
    elif (hour==15) or (hour==16) : return ("15H - 16H")
    elif (hour==17) or (hour==18) : return ("17H - 18H")
    elif (hour==19) or (hour==20) : return ("19H - 20H")
    else                          : return ("21H - 22H")
    
df.loc[:,'hourslice2']=df.loc[:,'Hour'].apply(hourslice2)   
z = pd.get_dummies(df['hourslice2'], drop_first=True)

In [ ]:
def season(month):
    if   (month==1) or (month==2) or (month==12) : return("winter")
    elif (month==3) or (month==4) or (month==5)  : return("spring")
    elif (month==6) or (month==7) or (month==8)  : return("summer")
    else                                         : return("autumn")
df.loc[:,'season']=df.loc[:,'Month'].apply(season)   
t = pd.get_dummies(df['season'], drop_first=True)   

In [ ]:
df = pd.concat([df, x,y,z,t], axis=1)

In [ ]:
def myfun(y): return(y.value // 10**9)
    
df.loc[:,'Date_sec']=df['Dates'].apply(myfun)

In [ ]:
df_t.loc[:,'DayOfWeek'] = df_t['DayOfWeek'].cat.reorder_categories(["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"], ordered=True)
x=pd.get_dummies(df_t['DayOfWeek'], drop_first=True)
y=pd.get_dummies(df_t['PdDistrict'], drop_first=True)
df_t.loc[:,'hourslice2']=df_t.loc[:,'Hour'].apply(hourslice2)   
z = pd.get_dummies(df_t['hourslice2'], drop_first=True)
df_t.loc[:,'season']=df_t.loc[:,'Month'].apply(season)   
t = pd.get_dummies(df_t['season'], drop_first=True)   

df_t = pd.concat([df_t, x,y,z,t], axis=1)

df_t.loc[:,'Date_sec']=df_t['Dates'].apply(myfun)

In [ ]:
import holidays
from datetime import date
us_holidays = holidays.UnitedStates()  # or holidays.US()

def isholiday(d) :
    return(int(d in us_holidays))
# example
int(isholiday('2001-01-01 23:53:00'))

df['Holiday']=df['Dates'].apply(isholiday)
df_t['Holiday']=df_t['Dates'].apply(isholiday)

df.dtypes

In [ ]:
# http://sanfranciscopolice.org/police-district-maps
# http://www.latlong.net/convert-address-to-lat-long.html

PoliceStations_raw=pd.DataFrame(
[['BAYVIEW'   ,-122.397981,37.729732],
 ['CENTRAL'   ,-122.409919,37.798732],
 ['INGLESIDE' ,-122.446215,37.724676],
 ['MISSION'   ,-122.422005,37.762849],
 ['NORTHERN'  ,-122.432467,37.780186],
 ['PARK'      ,-122.455287,37.767797],
 ['RICHMOND'  ,-122.464467,37.779928],
 ['SOUTHERN'  ,-122.389412,37.772380],
 ['TARAVAL'   ,-122.481500,37.743733],
 ['TENDERLOIN',-122.412899,37.783674]],columns=['PdDistrict','XX','YY'])
PoliceStations_raw

In [ ]:
df  =pd.merge(df  ,PoliceStations_raw,on='PdDistrict',how='left')
df_t=pd.merge(df_t,PoliceStations_raw,on='PdDistrict',how='left')

#df['Pd_Dist']=( (df['X']-df['XX']) **2 + (df['Y']-df['YY']) ** 2)
df.head()
# be careful pd.merge is resetting dtypes !!!!!!!
df.dtypes
df['Category'] = df_raw['Category'].astype('category')
df.head()

In [ ]:
df['Pd_Dist']=( (df['X']-df['XX']) **2 + (df['Y']-df['YY']) ** 2)
df['Pd_Dist']=np.sqrt(df['Pd_Dist'])

df_t['Pd_Dist']=( (df_t['X']-df_t['XX']) **2 + (df_t['Y']-df_t['YY']) ** 2)
df_t['Pd_Dist']=np.sqrt(df_t['Pd_Dist'])
df.head()

In [ ]:
# normalize test data with training data scaling !!!!! very important !!!! (easy mistake)

def norm_01(df1,df2):
    return( ( df1 - df1.min()) / (df1.max() - df1.min()) , ( df2 - df1.min()) / (df1.max() - df1.min()) ) 

In [ ]:
df['Xnorm'],df_t['Xnorm']=norm_01(df['X'],df_t['X'])
df['Ynorm'],df_t['Ynorm']=norm_01(df['Y'],df_t['Y'])
df['Hournorm'],df_t['Hournorm']=norm_01(df['Hour'],df_t['Hour'])
df['Date_secnorm'],df_t['Date_secnorm']=norm_01(df['Date_sec'],df_t['Date_sec'])
df['Pd_Distnorm'],df_t['Pd_Distnorm']= norm_01(df['Pd_Dist'],df_t['Pd_Dist'])

In [ ]:
# Adding Address Features
df_a=df.loc[:,'Address']
df_t_a = df_t.loc[:,'Address']


df['AV']=df_a.apply(lambda x: int(' AV' in x))
df['BL']=df_a.apply(lambda x: int(' BL' in x))
df['CR']=df_a.apply(lambda x: int(' CR' in x))
df['CT']=df_a.apply(lambda x: int(' CT' in x))
df['DR']=df_a.apply(lambda x: int(' DR' in x))
df['BL']=df_a.apply(lambda x: int(' BL' in x))
df['LN']=df_a.apply(lambda x: int(' LN' in x))
df['PL']=df_a.apply(lambda x: int(' PL' in x))
df['PZ']=df_a.apply(lambda x: int(' PZ' in x))
df['RD']=df_a.apply(lambda x: int(' RD' in x))
df['TR']=df_a.apply(lambda x: int(' TR' in x))
df['HWY']=df_a.apply(lambda x: int((' HY' in x) or ('HWY' in x)) )
df['WAY']=df_a.apply(lambda x: int((' WY' in x) or ('WAY' in x)) )
#i took out 'ST' because everything not above contains ST

df['IsIntersect'] = df_a.apply(lambda x: int('/' in x))

# same for test set

df_t['AV'] =df_t_a.apply(lambda x: int(' AV' in x))
df_t['BL'] =df_t_a.apply(lambda x: int(' BL' in x))
df_t['CR'] =df_t_a.apply(lambda x: int(' CR' in x))
df_t['CT'] =df_t_a.apply(lambda x: int(' CT' in x))
df_t['DR'] =df_t_a.apply(lambda x: int(' DR' in x))
df_t['BL'] =df_t_a.apply(lambda x: int(' BL' in x))
df_t['LN'] =df_t_a.apply(lambda x: int(' LN' in x))
df_t['PL'] =df_t_a.apply(lambda x: int(' PL' in x))
df_t['PZ'] =df_t_a.apply(lambda x: int(' PZ' in x))
df_t['RD'] =df_t_a.apply(lambda x: int(' RD' in x))
df_t['TR'] =df_t_a.apply(lambda x: int(' TR' in x))
df_t['HWY']=df_t_a.apply(lambda x: int((' HY' in x) or ('HWY' in x)) )
df_t['WAY']=df_t_a.apply(lambda x: int((' WY' in x) or ('WAY' in x)) )
#i took out 'ST' because everything not above contains ST

df_t['IsIntersect'] = df_t_a.apply(lambda x: int('/' in x))


In [ ]:
# be careful when adding the address features to the validation set.
# the validation set should have its features calculated in the same way than the test set 
# in version 4, we define the traindata and validdata at this stage, before calculating the logodds features
# that explains the over-optimistic evaluation of the logloss in version 3.

In [ ]:
import random
rows = random.sample(df.index, NObs//10)
# rows1 = rows[:len(rows)//2]
# rows2 = rows[len(rows)//2:]
# validdata = df.ix[rows1]
# testdata  = df.ix[rows2]

validdata =df.ix[rows]
# validdata =validdata.drop('Id',axis=1)

traindata = df.drop(rows)
#traindata=traindata.drop('Id',axis=1)

print(traindata.shape)
print(validdata.shape)


testdata = df_t

In [ ]:
df_a = traindata['Address']
df_v_a= validdata['Address']

address_train=pd.DataFrame(sorted(df_a.unique()),columns=['Address'])
address_valid=pd.DataFrame(sorted(df_v_a.unique()),columns=['Address'])
address_test =pd.DataFrame(sorted(df_t_a.unique()),columns=['Address'])


print('number of addresses in train set %d' % len(address_train))
print('number of addresses in valid set %d' % len(address_valid))
print('number of addresses in test  set %d' % len(address_test))

# categories=sorted(df["Category"].unique())
C_counts=traindata.groupby(["Category"]).size()
N_obs_train=len(traindata)
print(sum(C_counts))
print(len(traindata))
# A_C_counts=df.groupby(["Address","Category"]).size()

A_counts=traindata.groupby(["Address"]).size()
A_counts=pd.DataFrame(A_counts)
A_counts.columns=['count']
A_counts.head()


In [ ]:
check_address_valid=pd.merge(address_valid,A_counts,left_on='Address',right_index=True,how='left',indicator=True)
print('number of addresses in test set not present in the train set %d' % len(check_address_valid[check_address_valid['_merge']=='left_only']))
# 1563 new addresses in the test set, how many observations is that ?
check_address_valid

check_address_test=pd.merge(address_test,A_counts,left_on='Address',right_index=True,how='left',indicator=True)
print('number of addresses in test set not present in the train set %d' % len(check_address_test[check_address_test['_merge']=='left_only']))
# 1563 new addresses in the test set, how many observations is that ?
check_address_test


In [ ]:
address_test.head()
A_counts.head()

In [ ]:
df1 = pd.merge(traindata,A_counts,left_on='Address',how='left',right_index=True)
df1['one']=1

table = pd.pivot_table(df1, values='one',index=['Address'],
                 columns=['Category'], fill_value=0, aggfunc=np.sum)
Total=table.sum(axis=1)

def myfun(v):
    w=v.astype(float)
    return(np.divide(w,Total))
table=table.apply(myfun)
table
table.columns=["proba_"+str(x) for x in range(len(table.columns))]

df2=pd.merge(df1,table,left_on='Address',how='left',right_index=True).drop(['one'],axis=1)

In [ ]:
table.loc['Default',:]=np.array(C_counts/N_obs_train)
print(sum(table.loc['Default',:]))
table.tail()

In [ ]:
df_t_1 = pd.merge(df_t,check_address,on='Address',how='left')
df_t_1['Address'][df_t_1['_merge']=='left_only']='Default'
# number of obs in the test set with Default address
print(sum((df_t_1['Address']=='Default').astype(int)))

df_t_1

In [ ]:
df_t_2=pd.merge(df_t_1,table,left_on='Address',how='left',right_index=True)
print(df_t_2['count'].loc[0])
print(A_counts.loc['2000 Block of THOMAS AV'])

# set count to 1 for addresses which are present in test set, but not in the training set

# xxx=df_t_2[df_t_2['Address']=='Default']
# df_t_2['count'][df_t_2['Address']=='Default']
df_t_2['count'][df_t_2['Address']=='Default']=1 

df2   ['count']=df2.loc[:,'count']/N_obs_train
df_t_2['count']=df_t_2.loc [:,'count']/N_obs_train

In [ ]:
# leaving only normalized variables in the training set
df=df2.drop(['Id','Dates','Date_sec','Hour','season','Descript','PdDistrict','Resolution','Address','X','Y','XX','YY','hourslice2','DayOfWeek','Pd_Dist',
            'spring','summer','winter'],axis=1) 
# leaving only normalized variables in the test set
df_t=df_t_2.drop(['Id','_merge','Dates','Date_sec','Hour','season','PdDistrict','Address','X','Y','XX','YY','hourslice2','DayOfWeek','Pd_Dist',
                 'spring','summer','winter'],axis=1) 
print(len(df_t.columns))
print(len(df.columns))
# the 1 difference in column length is because train set includes Category.

In [ ]:
A_counts.head()
for x in df.columns: print x

In [ ]:
NObs = df.shape[0]
NObs//10
87798%128

In [ ]:
import random
rows = random.sample(df.index, NObs//10)
# rows1 = rows[:len(rows)//2]
# rows2 = rows[len(rows)//2:]
# validdata = df.ix[rows1]
# testdata  = df.ix[rows2]

validdata =df.ix[rows]
# validdata =validdata.drop('Id',axis=1)

traindata = df.drop(rows)
#traindata=traindata.drop('Id',axis=1)

print(traindata.shape)
print(validdata.shape)


testdata = df_t

In [ ]:
minidf= traindata.loc[1:5,:]
# minidf['hourslice2']
minidf

In [ ]:
column_names = minidf.columns
column_names

In [ ]:
num_variables = traindata.shape[1]-1 # substract 1 because we will drop the target variable later
num_variables

In [ ]:
traindata.dtypes

In [ ]:
num_Category=df['Category'].cat.categories.shape[0]
num_Category

In [ ]:
Categories=np.array(traindata.loc[:,'Category'].cat.categories)
Categories

In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [ ]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [ ]:
# prepare data for TensorFlow

num_var = num_variables   # 85
num_labels = num_Category # 39 categories

train_dataset = np.float32(np.array(traindata.drop(['Category'],axis=1)))
train_labels = traindata.loc[:,'Category']
train_labels= np.array(pd.get_dummies(train_labels.cat.rename_categories(np.arange(0,num_labels)) ))

valid_dataset = np.float32(np.array(validdata.drop(['Category'],axis=1)))
valid_labels = validdata.loc[:,'Category']
valid_labels= np.array(pd.get_dummies(valid_labels.cat.rename_categories(np.arange(0,num_labels)) ))

test_dataset = np.float32(np.array(df_t))
# test_labels = testdata.loc[:,'Category']
# test_labels= np.array(pd.get_dummies(test_labels.cat.rename_categories(np.arange(0,num_labels)) ))

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape)
test_dataset


In [ ]:
num_labels

In [ ]:
num_var

In [ ]:
# Adding regularization to the 1 hidden layer network
graph1 = tf.Graph()
batch_size = 128

num_steps=13001 #6501

import datetime
startTime = datetime.datetime.now() 

def define_and_run_batch(beta):
    
    num_RELU = 1024
    
    with graph1.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, num_var)) # remplace batch_size par None
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
    
      # in Kaggle competitions, we don't know the true labels of the test set
      tf_test_dataset = tf.constant(test_dataset)
  
      # Variables.
      weights_RELU = tf.Variable(
        tf.truncated_normal([num_var, num_RELU],stddev=0.1))
        
      # print(tf.shape(weights_RELU) )   
      #biases_RELU = tf.Variable(tf.zeros([num_RELU]))
      biases_RELU = tf.Variable(tf.random_normal([num_RELU],stddev=0.1))  
        
      weights_layer1 = tf.Variable(
        tf.truncated_normal([num_RELU, num_labels],stddev=0.1))
      #biases_layer1 = tf.Variable(tf.zeros([num_labels]))
      biases_layer1 = tf.Variable(tf.random_normal([num_labels],stddev=0.1))
  
      # Training computation.
      logits_RELU = tf.matmul(tf_train_dataset, weights_RELU) + biases_RELU
      RELU_vec = tf.nn.relu(logits_RELU)
      logits_layer = tf.matmul(RELU_vec, weights_layer1) + biases_layer1                  
      # loss = tf.reduce_mean(
      #        tf.nn.softmax_cross_entropy_with_logits(logits_layer, tf_train_labels))
      cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits_layer, tf_train_labels,name="cross_entropy")
      l2reg = tf.reduce_sum(tf.square(weights_RELU))+tf.reduce_sum(tf.square(weights_layer1))
      
      loss = tf.reduce_mean(cross_entropy+beta*l2reg)
      logloss = tf.reduce_mean(cross_entropy)
    
  # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
  
      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits_layer)
      
      valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu((tf.matmul(tf_valid_dataset, weights_RELU) + biases_RELU)),weights_layer1)+biases_layer1)
        
      # in Kaggle competitions, we don't know the true labels of the test set     
      test_prediction =tf.nn.softmax(
        tf.matmul(tf.nn.relu((tf.matmul(tf_test_dataset, weights_RELU) + biases_RELU)),weights_layer1)+biases_layer1)
                         
    with tf.Session(graph=graph1) as session:
 
      tf.initialize_all_variables().run()
      print("Initialized")
      for step in range(num_steps):
        
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch. 
        batch_data = train_dataset[offset:(offset + batch_size), :]
        
        batch_labels = train_labels[offset:(offset + batch_size)]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        #
        if (step==num_steps-1):
            _, l, predictions_train,predictions_test = session.run(
              [optimizer, logloss,train_prediction,test_prediction], feed_dict=feed_dict)
        else:
            _, l, predictions_train = session.run(
              [optimizer, logloss,train_prediction], feed_dict=feed_dict)
            
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions_train, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      
      # test_acc = accuracy(test_prediction.eval(), test_labels)
      # print("Test accuracy: %.1f%%" % test_acc)

      print('loss=%s' % l)
        
    x = datetime.datetime.now() - startTime
    print(x)
    return(predictions_test)
    
    
RES=define_and_run_batch(0.0001)

In [ ]:
# 4 layer network

graph1 = tf.Graph()
batch_size = 128

num_steps=39001 #6501

import datetime
startTime = datetime.datetime.now() 

def define_and_run_batch_4_layer():
    
    

    import math as math

    initial_learning_rate_value = 0.5
    beta1 = 0.001
    beta2 = 0.001
    beta3 = 0.001
    beta4 = 0.001
    beta5 = 0.001

    graph2 = tf.Graph()
    with graph2.as_default():

       # Input data. For the training data, we use a placeholder that will be fed
       # at run time with a training minibatch.
       tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, num_var))
       tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        
       tf_valid_dataset = tf.constant(valid_dataset)
       tf_test_dataset = tf.constant(test_dataset)    
    
       # learning rate decay
       global_step = tf.Variable(0)
       learning_rate = tf.train.exponential_decay(initial_learning_rate_value, global_step, 1, 0.9999)

       # Hidden layer 1
       h1_size = 2048  
       weights_h1 = tf.Variable(tf.truncated_normal([num_var, h1_size], stddev=math.sqrt(2.0/(num_var))))
       biases_h1 = tf.Variable(tf.zeros([h1_size]))
       h1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h1) + biases_h1)

       # Hidden layer 2
       h2_size = 1024
       weights_h2 = tf.Variable(tf.truncated_normal([h1_size, h2_size], stddev=math.sqrt(2.0/h1_size)))
       biases_h2 = tf.Variable(tf.zeros([h2_size]))
       h2 = tf.nn.relu(tf.matmul(h1, weights_h2) + biases_h2)

       # Hidden layer 3
       h3_size = 512
       weights_h3 = tf.Variable(tf.truncated_normal([h2_size, h3_size], stddev=math.sqrt(2.0/h2_size)))
       biases_h3 = tf.Variable(tf.zeros([h3_size]))
       h3 = tf.nn.relu(tf.matmul(h2, weights_h3) + biases_h3)

       # Hidden layer 4
       h4_size = 256
       weights_h4 = tf.Variable(tf.truncated_normal([h3_size, h4_size], stddev=math.sqrt(2.0/h3_size)))
       biases_h4 = tf.Variable(tf.zeros([h4_size]))
       h4 = tf.nn.relu(tf.matmul(h3, weights_h4) + biases_h4)

       # Output layer
       weights_o = tf.Variable(tf.truncated_normal([h4_size, num_labels], stddev=math.sqrt(2.0/h4_size)))
       biases_o = tf.Variable(tf.zeros([num_labels]))
    
       # Training computation.
       logits = tf.matmul(h4, weights_o) + biases_o
       # loss = tf.reduce_mean(
       #     tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + ((beta1 * tf.nn.l2_loss(weights_h1)) + (beta2 * tf.nn.l2_loss(weights_h2)) 
       #        + (beta3 * tf.nn.l2_loss(weights_h3)) + (beta4 * tf.nn.l2_loss(weights_h4)) + (beta5 * tf.nn.l2_loss(weights_o)))
       
       cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels,name="cross_entropy")
       logloss = tf.reduce_mean(cross_entropy)
    
       loss = logloss + ((beta1 * tf.nn.l2_loss(weights_h1)) + (beta2 * tf.nn.l2_loss(weights_h2)) + (beta3 * tf.nn.l2_loss(weights_h3)) 
                         + (beta4 * tf.nn.l2_loss(weights_h4)) + (beta5 * tf.nn.l2_loss(weights_o)))
       
    
       # Optimizer.
       optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

       # Predictions for the training, validation, and test data.
       train_h1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h1) + biases_h1)
       train_h2 = tf.nn.relu(tf.matmul(train_h1, weights_h2) + biases_h2)
       train_h3 = tf.nn.relu(tf.matmul(train_h2, weights_h3) + biases_h3)
       train_h4 = tf.nn.relu(tf.matmul(train_h3, weights_h4) + biases_h4)
       train_logits = tf.matmul(train_h4, weights_o) + biases_o
       train_prediction = tf.nn.softmax(train_logits)
  
       valid_h1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h1) + biases_h1)
       valid_h2 = tf.nn.relu(tf.matmul(valid_h1, weights_h2) + biases_h2)
       valid_h3 = tf.nn.relu(tf.matmul(valid_h2, weights_h3) + biases_h3)
       valid_h4 = tf.nn.relu(tf.matmul(valid_h3, weights_h4) + biases_h4)
       valid_logits = tf.matmul(valid_h4, weights_o) + biases_o
       valid_prediction = tf.nn.softmax(valid_logits)
    
       test_h1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_h1) + biases_h1)
       test_h2 = tf.nn.relu(tf.matmul(test_h1, weights_h2) + biases_h2)
       test_h3 = tf.nn.relu(tf.matmul(test_h2, weights_h3) + biases_h3)
       test_h4 = tf.nn.relu(tf.matmul(test_h3, weights_h4) + biases_h4)
       test_logits = tf.matmul(test_h4, weights_o) + biases_o
       test_prediction = tf.nn.softmax(test_logits)
  
        
   

# executing the graph     
        
        
    with tf.Session(graph=graph2) as session:
 
      tf.initialize_all_variables().run()
      print("Initialized")
      for step in range(num_steps):
        
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch. 
        batch_data = train_dataset[offset:(offset + batch_size), :]
        
        batch_labels = train_labels[offset:(offset + batch_size)]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        #
        if (step==num_steps-1):
            _, l, predictions_train,predictions_test = session.run(
              [optimizer, logloss,train_prediction,test_prediction], feed_dict=feed_dict)
        else:
            _, l, predictions_train = session.run(
              [optimizer, logloss,train_prediction], feed_dict=feed_dict)
            
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions_train, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      
      # test_acc = accuracy(test_prediction.eval(), test_labels)
      # print("Test accuracy: %.1f%%" % test_acc)

      print('loss=%s' % l)
        
    x = datetime.datetime.now() - startTime
    print(x)
    return(predictions_test)
    
    
RES=define_and_run_batch_4_layer()

In [ ]:
RES.shape

In [ ]:
train_dataset = np.float32(np.array(traindata.drop(['Category'],axis=1)))
train_labels = traindata.loc[:,'Category']
train_labels=train_labels.cat.rename_categories(np.arange(0,num_labels))
train_labels
pd.get_dummies(train_labels).head()

In [ ]:
# use sklearn one-hot-encoder only if using sklearn functions
# http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [ ]:
###### XGBOOST ######

import datetime
startTime = datetime.datetime.now() 

import xgboost as xgb
data_train   = np.array(traindata.drop('Category',axis=1))
labels_train = np.array(traindata['Category'].cat.codes)

data_valid   = np.array(validdata.drop('Category',axis=1))
labels_valid = np.array(validdata['Category'].cat.codes)

weights_train = np.ones(len(labels_train))
weights_valid  = np.ones(len(labels_valid ))

dtrain = xgb.DMatrix( data_train, label=labels_train,weight = weights_train)
dvalid  = xgb.DMatrix( data_valid , label=labels_valid ,weight = weights_valid )




param = {'bst:max_depth':5, 'bst:eta':0.5, # eta [default=0.3]
         #'min_child_weight':1,'gamma':0,'subsample':1,'colsample_bytree':1,'scale_pos_weight':0, # default
         # max_delta_step:0 # default
         'min_child_weight':6,'scale_pos_weight':0.1, 'max_delta_step':1
         'subsample':0.8,'colsample_bytree':0.8,
         'silent':1, 'objective':'multi:softprob' }


param['nthread'] = 4
param['eval_metric'] = 'mlogloss'
param['lambda'] = 1
param['num_class']=39

# evallist  = [(dvalid,'eval'),(dtrain,'train')] # if there is a validation set
evallist  = [(dtrain,'train')]                   # if there is no validation set

plst = param.items()
plst += [('eval_metric', 'ams@0')]

num_round = 10
bst = xgb.train( plst, dtrain, num_round, evallist ) # early_stopping_rounds=10 # when there is a validation set

bst.save_model('0001.model')

# dump model
bst.dump_model('dump.raw.txt')
# dump model with feature map
# bst.dump_model('dump.raw.txt','featmap.txt')

x = datetime.datetime.now() - startTime
print(x)

In [ ]:
minidf['Category'].cat.codes

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
xgb.plot_importance(bst)
xgb.plot_tree(bst, num_trees=2)
#xgb.to_graphviz(bst, num_trees=2)

In [ ]:
data_test   = np.array(df_t)

dtest = xgb.DMatrix( data_test)

In [ ]:
predXGB = bst.predict(dtest)


In [ ]:
df_submit=pd.read_csv('/Users/francois-guillaume.rideau/Documents/Kaggle/Kaggle-SF/sampleSubmission.csv',dtype=float)
df_submit.head()

In [ ]:
df_submit.shape
type(df_submit)

In [ ]:
for i in range (1,40): 
    df_submit[df_submit.columns[i]]=predXGB[:,i-1] # for XGBoost
    # df_submit[df_submit.columns[i]]=RES[:,i-1]   # for TensorFlow
df_submit['Id']=df_submit['Id'].astype(int)



In [ ]:
df_submit.head()

In [ ]:
filename = '/Users/francois-guillaume.rideau/Documents/Kaggle/Kaggle-SF/submitXGB5.csv'
df_submit.to_csv(filename,index=False)

In [ ]:
df_verify=pd.read_csv(filename,dtype=float)
df_verify.head()

In [ ]:
z=df_verify.describe()
z

In [ ]:
# checking if the mean of predicted probabilities match with the distribution in the training set
zz=z.loc['mean',:].sort_values(ascending=False)
zz

In [ ]:
# distribution of Category in the training set
df_raw['Category'].value_counts()/877982

In [ ]:
predXGB_train = bst.predict(dtrain)


In [ ]:
predXGB_train=pd.DataFrame(predXGB_train)
predXGB_train

In [ ]:
predXGB_train.columns=df_raw['Category'].cat.categories

In [ ]:
z=predXGB_train.describe()

In [ ]:
z

In [ ]:
z.ix[1]

In [ ]:
df_raw['Voie']='Other'

In [ ]:
df_raw['Voie']=d

In [ ]:
df1=pd.DataFrame([[0,'CENTRAL'],[1,'TARAVAL'],[3,'CENTRAL'],[2,'BAYVIEW']])
df1.columns = ['Index','PdDistrict']
df1

In [ ]:
np.sqrt(880000*39)

In [ ]:
df.columns